<a href="https://colab.research.google.com/github/georgsmeinung/rn1-perceptron/blob/main/RN_Clase_3_Actividad_en_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

def prepare_data(df):
    """Prepara los datos para el entrenamiento."""
    # Convertir la columna 'Clase' a binaria: Tipo2=1, otros=0
    df['Target'] = df['Clase'].apply(lambda x: 1 if x == 'Tipo2' else 0)

    # Separar características (X) y objetivo (y)
    features = ['Area', 'Perimetro', 'Compacidad', 'LongNucleo', 'AnchoNucleo', 'Asimetria', 'LongSurco']
    X = df[features]
    y = df['Target']

    # Escalar las características (importante para el descenso de gradiente)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y.values, features, scaler

def build_model(input_dim):
    """
    Construye el modelo de una sola neurona (perceptrón)
    con activación sigmoide.
    """
    model = Sequential()

    # Se añade la única capa densa (una neurona)
    # units=1: Una sola neurona en la capa.
    # input_dim=input_dim: Número de características de entrada (7).
    # activation='sigmoid': Función de activación sigmoide (0 a 1).
    model.add(Dense(units=1,
                    input_dim=input_dim,
                    activation='sigmoid'))

    # Compilar el modelo
    # optimizer='sgd': Descenso de Gradiente Estocástico.
    # loss='mean_squared_error': Error Cuadrático Medio (ECM).
    optimizer = SGD()
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    return model

def train_and_evaluate(model, X, y):
    """Entrena y evalúa el modelo."""
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print("Iniciando entrenamiento...")
    # Se entrena el modelo
    history = model.fit(X_train, y_train,
                        epochs=100,       # Número de pasadas sobre los datos
                        batch_size=8,     # Tamaño del lote para SGD
                        validation_data=(X_test, y_test),
                        verbose=0)        # No mostrar el progreso de cada época

    print("Entrenamiento completado.")

    # Evaluar el modelo con los datos de prueba
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nEvaluación en datos de prueba:")
    print(f"  Error Cuadrático Medio (ECM): {loss:.4f}")
    print(f"  Precisión (Accuracy): {accuracy * 100:.2f}%")

    # Obtener y mostrar los pesos finales
    weights = model.get_weights()
    kernel_weights = weights[0] # Pesos W1 a W7
    bias_weight = weights[1]    # Peso W0 (sesgo)

    return kernel_weights, bias_weight

In [3]:
# --- Ejecución Principal ---
try:
    # 1. Cargar datos
    df_semillas = pd.read_csv( DATOS_DIR+'semillas.csv')

    # 2. Preparar datos
    X_scaled, y, feature_names, _ = prepare_data(df_semillas)

    # 3. Construir modelo
    # Se pasa la cantidad de características de entrada (7)
    model = build_model(input_dim=X_scaled.shape[1])
    print("Modelo construido:")
    model.summary()

    # 4. Entrenar y obtener pesos
    kernel_weights, bias_weight = train_and_evaluate(model, X_scaled, y)

    # 5. Mostrar pesos finales
    print("\nPesos obtenidos tras el entrenamiento:")
    print("=" * 35)
    print(f"Pesos de características (W1 a W7):")
    for feature, weight in zip(feature_names, kernel_weights.flatten()):
        print(f"  - {feature}: {weight:.6f}")

    print(f"\nPeso del sesgo (W0 o bias):")
    print(f"  - Bias: {bias_weight[0]:.6f}")

except FileNotFoundError:
    print("Error: No se encontró el archivo 'semillas.csv'.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Modelo construido:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             8 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8 (32.00 B)

 Trainable params: 8 (32.00 B)

 Non-trainable params: 0 (0.00 B)

Iniciando entrenamiento...
Entrenamiento completado.

Evaluación en datos de prueba:
  Error Cuadrático Medio (ECM): 0.0253
  Precisión (Accuracy): 97.62%

Pesos obtenidos tras el entrenamiento:
Pesos de características (W1 a W7):
  - Area: 1.116188
  - Perimetro: -0.185349
  - Compacidad: 0.132139
  - LongNucleo: 0.056312
  - AnchoNucleo: 0.361469
  - Asimetria: 0.288671
  - LongSurco: 1.219899

Peso del sesgo (W0 o bias):
  - Bias: -0.695373


In [4]:
# Create a DataFrame for the new data point
new_data = {
    'Area': [15.03],
    'Perimetro': [14.77],
    'Compacidad': [0.8658],
    'LongNucleo': [5.702],
    'AnchoNucleo': [3.212],
    'Asimetria': [1.933],
    'LongSurco': [5.439]
}
new_df = pd.DataFrame(new_data)

# Scale the new data using the same scaler used for training
# We need to retrieve the scaler from the prepare_data function or re-fit it on the original data
# For simplicity here, let's assume we have access to the scaler object.
# If not, we would need to modify prepare_data to return the scaler.
# Assuming the scaler is returned by prepare_data:
# X_scaled, y, feature_names, scaler = prepare_data(df_semillas)
# And then use:
# new_data_scaled = scaler.transform(new_df)

# Since the scaler was not returned, let's re-fit it on the original data for demonstration
# In a real scenario, you should save and load the scaler.
_, _, _, scaler = prepare_data(df_semillas) # Re-fit scaler on original data to get the scaler object
new_data_scaled = scaler.transform(new_df)


# Predict the class of the new data point
prediction = model.predict(new_data_scaled)

# The output of the sigmoid is a probability (between 0 and 1).
# We can interpret values close to 1 as belonging to class 1 (Tipo2)
# and values close to 0 as belonging to class 0 (not Tipo2).
# A common threshold is 0.5.
predicted_class = 1 if prediction[0][0] > 0.5 else 0

print(f"The predicted probability for class Tipo2 is: {prediction[0][0]:.4f}")
print(f"Based on a threshold of 0.5, the predicted class is: {predicted_class}")

# Map the predicted class back to the original class names if needed
# class_names = {0: 'Not Tipo2', 1: 'Tipo2'}
# predicted_class_name = class_names[predicted_class]
# print(f"The predicted class name is: {predicted_class_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
The predicted probability for class Tipo2 is: 0.2723
Based on a threshold of 0.5, the predicted class is: 0
